WORKSHOP FINALE
===============

In [1]:
from pyplasm import*
from larlib import*
import math
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.007323 seconds


In [2]:
def perimetro(): 
    with open("perimetro.lines", "rb") as file:
        file_reader = csv.reader(file, delimiter=",")
        chiavi = {} ### diizionario delle chiavi
        nodi = [] ### array con i vertici
        archi= [] ### array con gli archi, ovvero gli spigoli
        i = 0 ### inizializzo un contatore
    
        for righe in file_reader:
            riga0 = math.floor(float(righe[0])*100)/100
            riga1 = math.floor(float(righe[1])*100)/100
            riga2 = math.floor(float(righe[2])*100)/100
            riga3 = math.floor(float(righe[3])*100)/100
        
            if not(riga0,riga1) in chiavi.keys():
                chiavi[(riga0,riga1)] = i
                nodi.append((riga0,riga1))
                i = i+1
            
            if not(riga2,riga3) in chiavi.keys():
                chiavi[(riga2,riga3)] = i
                nodi.append((riga2,riga3))
                i = i+1
            archi.append((chiavi[(riga0,riga1)],(chiavi[(riga2,riga3)])))
        pavimento = T(2)(450)(CUBOID([850,400]))
        corpoVilla = MULTEXTRUDE(OFFSET([20, 20])(STRUCT(MKPOLS([nodi,archi]))))(600)
    return TEXTURE("intonaco.jpg")(STRUCT([pavimento,corpoVilla]))

In [10]:
def tettoTriangolare1(depth,width):
    return TEXTURE("tetto.jpg")(S([1,2,3])([22,28.34,30])((ROTATE([1, 2])(PI)(ROTATE([2,3])(PI/2)(
        MULTEXTRUDE(MKPOL([[[0, 0],[width/2,width/4],[width,0]],[[1,2,3]],None]))(depth))))))

In [11]:
def sopra():
    return STRUCT([tettoTriangolare1(10,22)])


In [315]:
def comignoli():
    
    base = TEXTURE("intonaco.jpg")(T([1,2])([2.5,2.5])(CUBOID([10,10,20])))
    
    tettino = TEXTURE("intonaco.jpg")(T(3)(20)(CUBOID([15,15,2])))
    
    rialzo = TEXTURE("intonaco.jpg")(CUBOID([2,1.5,2]))
    
    rialziDestra = TEXTURE("intonaco.jpg")(
        STRUCT([T([1,2,3])([1,1,22])(rialzo),T([1,2,3])([1,6.75,22])(rialzo),T([1,2,3])([1,12.5,22])(rialzo)]))
    
    rialziSinistra = TEXTURE("intonaco.jpg")(
        STRUCT([T([1,2,3])([12,1,22])(rialzo),T([1,2,3])([12,6.75,22])(rialzo),T([1,2,3])([12,12,22])(rialzo)]))
    
    tetto = TEXTURE("intonaco.jpg")(T(3)(24)(CUBOID([15,15,1])))
    
    comignolo = S([1,2,3])([4,4,4])(STRUCT([base,tettino,rialziDestra,rialziSinistra,tettino,tetto]))
    
    return STRUCT([TEXTURE("intonaco.jpg")(comignolo)])

In [316]:
VIEW(STRUCT([TOP([perimetro(),TOP([TEXTURE("tetto.jpg")(CUBOID([850,400])),
                                   LEFT([comignoli(),RIGHT([comignoli(),STRUCT([tettoTriangolare1(10,22)])])])])])]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181837a810> >

In [317]:
def SEMISPHERE (radius):
        def GSPHERE0 (subds):
            N , M = subds
            domain = Hpc(Grid([N*[PI/N],M*[PI/M]]))
            domain = MAT([[1,0,0,0],[-PI/2,1,0,0],[-PI,0,1,0],[0,0,0,1]])(domain)
            fx  = lambda p: radius * math.cos(p[0]) * math.sin  (p[1])
            fy  = lambda p: radius * math.cos(p[0]) * math.cos (p[1])
            fz  = lambda p: radius * math.sin(p[0])
            ret = GMAP([fx, fy, fz])(domain)
            return ret
        return GSPHERE0
def facciata():
        cilindretto=(CUBOID([20,30,120]))
        sferetta =R([1,3])(-PI/2)(SEMISPHERE(24)([500,320]))
        insieme = TOP([cilindretto,sferetta])
        return (T([1,3,2])([-10,450,459])(STRUCT([insieme, T(1)(3)(insieme), T(1)(9)(insieme)])))
def fronte():
    quadrato =TOP([TEXTURE("marrone.jpg")(T([1,3,2])([130,100,450])(CUBOID([50,30,150]))),tettoTriangolare1(2,2.4)])
    return STRUCT([quadrato,T([1])([100])(quadrato),T([1])([600])(quadrato),T([1])([700])(quadrato)])
def columna(dm,h,h_base):
    
    cylndr = COMP([JOIN,TRUNCONE([dm/2,0.8*(dm/2),h])])(24)
    
    torus_bot = COMP([JOIN,TORUS([dm/12,dm/2])])([8,24])
    
    torus_top = COMP([JOIN,TORUS([0.8*(dm/12),0.8*(dm/2)])])([8,27])
    
    base = COMP([T([1,2])([7*(dm/-12),7*(dm/-23)]),CUBOID])([7*(dm/6),7*(dm/6),h_base])
    
    base_top = COMP([T([1,2])([7*(dm/-12),7*(dm/-12)]),CUBOID])([7*(dm/6),7*(dm/6),dm/6])
    
    capital = SUM([COMP([JOIN,TRUNCONE([0.8*dm/2, 1.2*dm/2,h/8])])(4),COMP([R([1,2])(PI/4),JOIN,TRUNCONE([0.8*dm/2,1.2*dm/2,
                                                                                                         h/8])])(4)]) 
    return (TOP([TOP([TOP([TOP([TOP([base, torus_bot]), cylndr]), torus_top]), capital]), base_top]))
def colonnato():
    c=STRUCT([columna(30,350,30),T([1])([50])(columna(30,350,30)),T([1])([150])(columna(30,350,30)),T([1])([200])(columna(30,350,30))])
    return c
def piramide(depth,width,height):
    
    _width = float(width)
    
    _scale = float(depth)/_width
    
    _height = float(height)
    
    return SCALE(2)(_scale)(ROTATE([1,2])(PI/4)(CONE([_width*(1/SQRT(2)),_height])(4)))

def scala():
    
    corrimanoDestro = TEXTURE("intonaco.jpg")(T([1,2,3])([330,1070,127.5])(R([2,3])(-PI)(DIFFERENCE([CUBOID([20,150,130]),
                                                                             piramide(400,10000,100)]))))
    
    corrimanoSinistro = TEXTURE("intonaco.jpg")(T([1,2,3])([500,1070,127.5])(R([2,3])(-PI)(DIFFERENCE([CUBOID([20,150,130]),
                                                                             piramide(400,10000,100)]))))
    return STRUCT([T([1,2,3])([350,1070,102])(R([2,3])(-PI)(TEXTURE("marrone.jpg")(DIFFERENCE([CUBOID([160,150,103]),
                                                                        piramide(300,70000,100)])))),corrimanoDestro,corrimanoSinistro])
def partesuperiore():
    return STRUCT([TOP([TEXTURE("tetto.jpg")(CUBOID([850,400])),LEFT([comignoli(),RIGHT([comignoli(),STRUCT([tettoTriangolare1(10,22)])])])])]) 
                                                                

In [602]:

def elementifacciata():
    return STRUCT([T([1,3])([-65,40])(fronte()), T([1,2,3])([300,490,0])(TOP([T(1)(10)(colonnato()),TEXTURE("intonaco.jpg")(CUBOID([550,5,140]))]))
,T([2,3])([1200,100])(R([2,3])(PI)(scala()))])

In [603]:
def paretianteriori():
    pavimento =TEXTURE("intonaco.jpg")(CUBOID([850,200]))
    parete = TEXTURE("intonaco.jpg")(CUBOID([8,200,600]))
    return STRUCT([TOP([STRUCT([T(2)(-10)(TEXTURE("intonaco.jpg")(CUBOID([300,10,600]))),T([2,1])([-10,600])(TEXTURE("intonaco.jpg")(CUBOID([270,10,600]))),RIGHT([parete,LEFT([parete,pavimento])])]),TEXTURE("tetto.jpg")(CUBOID([860,250]))])])
    

In [604]:
def porta():
    quadrato =TOP([TEXTURE("marrone.jpg")(T([1,3,2])([330,0,450])(CUBOID([150,30,300]))),tettoTriangolare1(7,7.4)])
    return STRUCT([T([1])([50])(quadrato)])

In [605]:
def porticato():
    pavimento =TEXTURE("intonaco.jpg")(CUBOID([850,550]))
    parete = TEXTURE("intonaco.jpg")(CUBOID([8,200,100]))
    return STRUCT([T(2)(-10)(TEXTURE("intonaco.jpg")(CUBOID([350,10,100]))),T([2,1])([-10,520])(TEXTURE("intonaco.jpg")(CUBOID([350,10,100]))),RIGHT([parete,LEFT([parete,pavimento])])])

In [606]:
def partesuperiore():
    return (T([2,3])([0,0])(STRUCT([TOP([perimetro(),T(2)(600)(TOP([TEXTURE("tetto.jpg")(CUBOID([860,500])),
                                   LEFT([comignoli(),RIGHT([comignoli(),STRUCT([tettoTriangolare1(10,22)])])])]))])])))

In [ ]:
def prato():
    
    prato = TEXTURE("prato.jpg")(S([1,2])([2,2])(CUBOID([1600,1400])))

    return STRUCT([prato])

In [ ]:
VIEW(STRUCT([T(2)(250)(paretianteriori()),partesuperiore(),T(2)(-250)(elementifacciata()),T(2)(50)(porticato()),porta()]),prato())

Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
Child 3
